Question Answering Program  on context not extractive(RAG without information retrival)


Scores:

    Average Cosine Similarity Score: 0.8703485468612207

    BLEU Score: 0.44343344508508914
        Needs Improvement: BLEU score could benefit from enhancement to improve n-gram overlap.

    ROUGE Scores:
        rouge-1:
            Recall (r): 0.4931050996492532
            Precision (p): 0.5062838536207949
            F1-score (f): 0.48421650710726766
        rouge-2:
            Recall (r): 0.2867267329318735
            Precision (p): 0.29386559356758024
            F1-score (f): 0.28114994067762966
            Needs Improvement: ROUGE-2 score is lower, indicating room for improvement in bigram overlap.
        rouge-l:
            Recall (r): 0.4929319008833392
            Precision (p): 0.5060474222029537
            F1-score (f): 0.4840231909599921

In [ ]:
!pip install datasets
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
!pip install accelerate>=0.21.0
!pip install transformers[torch]

SQuAD v2 (Stanford Question Answering Dataset v2)




    Strengths:
        Widely recognized benchmark in QA.
        Contains both answerable and unanswerable questions, handling ability to handle uncertainty.

Natural Questions (NQ)

    Strengths:
        Real-world questions from Google search, with detailed, comprehensive answers.
        Emphasizes handling of both short and long answers.

HotPot QA:

  Contains multi-hop questions that require reasoning over multiple documents to arrive at an answe




TriviaQA

    Strengths:
        Focuses on more challenging, trivia-based questions with complex answers.
        Covers a wide range of topics and domains.

NewsQA

    Strengths:
        Derived from news articles, offering narrative-driven contexts and answers.
        Useful for demonstrating the ability to understand and generate responses from structured journalistic content.

Will begin with important imports and functions to format data sets, the project will begin with SQuAD_v2 and other will be added afterward

In [ ]:
from datasets import load_dataset
from transformers import  T5ForConditionalGeneration, Trainer, TrainingArguments

Helper Function to load and then format a dataset

In [ ]:
def load_and_format_dataset(name, format_func, split='train'):
    """
    Loads and formats a dataset using a specified formatting function.

    Parameters:
    - name (str): The name of the dataset to load.
    - format_func (function): A function that defines how to format each example in the dataset.
    - split (str): The split of the dataset to load ('train', 'validation', or 'test'). Defaults to 'train'.

    Returns:
    - A formatted dataset with the specified split.
    """
    dataset = load_dataset(name)

    if split in dataset:
        formatted_dataset = dataset[split].map(format_func, remove_columns=dataset[split].column_names)
        return formatted_dataset
    else:
        raise ValueError(f"No {split} split found in the {name} dataset")

Formatting datasets:
using T5 so ill format it in accordance to that other models will need diffrent formatting (obviously)

i provide 2 approach each with its pros and cos ill use approach 1 as it both give number of answers and let the dataset size stay the same size , but an answer should be chosen from the collection to provide for the user afterward
approach 2 let the dataset stay the same size but give only 1 answer


In [ ]:
def format_squad_all_answers(example):
    # Join all possible answers into a single string separated by " | "
    all_answers = " | ".join(example['answers']['text'])
    return {
        'input_text': f"question: {example['question']} context: {example['context']}",
        'target_text': all_answers
    }
def format_squad_best_answer(example):
    # Choose the first available answer as the target text
    best_answer = example['answers']['text'][0] if example['answers']['text'] else 'No answer'
    return {
        'input_text': f"question: {example['question']} context: {example['context']}",
        'target_text': best_answer
    }


In [ ]:
squad_train = load_and_format_dataset('squad_v2', format_squad_best_answer, split='train')
squad_val = load_and_format_dataset('squad_v2', format_squad_best_answer, split='validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

Empty Spot to combine datasets if add more then SQuaAD_v2

In [ ]:
#combine HERE

Now that the dataset  is formated as wanted we need to tokenize it

In [ ]:
from transformers import T5TokenizerFast


model_name = 't5-small'

tokenizer = T5TokenizerFast.from_pretrained(model_name)

def tokenize(example):
    input_encodings = tokenizer(example['input_text'],
                                padding='max_length',
                                truncation=True, max_length=512)
    target_encodings = tokenizer(example['target_text'],
                                 padding='max_length',
                                 truncation=True, max_length=64)
    labels = target_encodings['input_ids']
    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]  # Ignore padding tokens in loss
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': labels}

# Tokenize combined training dataset
tokenized_train = squad_train.map(tokenize, remove_columns=squad_train.column_names, batched=True)

# Tokenize combined validation dataset
tokenized_val = squad_val.map(tokenize, remove_columns=squad_val.column_names, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

Now all that left is to train the T5 model on our dataset
We will create every needed part for it now using pytroch

For Now a simple training using HuggingFace

In [ ]:
# Insert out Put Directory HERE:
output_dir=""

Mounted at /content/drive


In [ ]:
from transformers import (T5ForConditionalGeneration, Trainer,
                         TrainingArguments, DataCollatorWithPadding)

# Initialize the tokenizer and model
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",   # Evaluate every `eval_steps` steps
    save_strategy="steps",         # Save checkpoint every `save_steps` steps
    save_steps=500,                # Save every 500 steps
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    logging_dir=f"{output_dir}/logs",
    logging_steps=500,
    gradient_accumulation_steps=4,
    warmup_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)


trainer.train()
#  trainer.train(resume_from_checkpoint=True)# when continueing from a checkpoint

# Manually save the final model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss
11000,0.034000,0.032780
11500,0.034300,0.032525
12000,0.034500,0.032603


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('/content/drive/My Drive/t5_model_Q_Answering/tokenizer_config.json',
 '/content/drive/My Drive/t5_model_Q_Answering/special_tokens_map.json',
 '/content/drive/My Drive/t5_model_Q_Answering/spiece.model',
 '/content/drive/My Drive/t5_model_Q_Answering/added_tokens.json',
 '/content/drive/My Drive/t5_model_Q_Answering/tokenizer.json')

In [ ]:
from tqdm import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
squad_test = load_and_format_dataset('squad', format_squad_best_answer, split='validation')

# Function to generate answers
def generate_answer(question):
    inputs = tokenizer.encode("question: " + question, return_tensors="pt").to(device)  # Move to device here
    outputs = model.generate(inputs)
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_answer


predicted_answers = []
for question in tqdm(squad_test['input_text'], desc="Evaluating"):
    predicted_answer = generate_answer(question)
    predicted_answers.append(predicted_answer)


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Evaluating:   0%|          | 0/10570 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Evaluating: 100%|██████████| 10570/10570 [10:48<00:00, 16.31it/s]


In [ ]:
!pip install rouge

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge



reference_answers = squad_test['target_text']  # List of ground truth answers from test set
generated_answers = predicted_answers  # List of answers generated by the model

# Filter out empty strings from generated_answers and ensure alignment
filtered_generated_answers = []
aligned_reference_answers = []

for gen_ans, ref_ans in zip(generated_answers, reference_answers):
    if gen_ans.strip():  # Check if gen_ans is not empty after stripping whitespace
        filtered_generated_answers.append(gen_ans)
        aligned_reference_answers.append(ref_ans)

# Ensure there are reference answers to compare against
if not aligned_reference_answers:
    raise ValueError("No reference answers available for evaluation.")

# Calculate BLEU score
bleu_score = corpus_bleu([[ref] for ref in aligned_reference_answers], filtered_generated_answers)

# Calculate ROUGE score
rouge = Rouge()
scores = rouge.get_scores(filtered_generated_answers, aligned_reference_answers, avg=True)


# Print evaluation results
print(f"BLEU Score: {bleu_score}")
print(f"ROUGE Scores: {scores}")



BLEU Score: 0.44343344508508914
ROUGE Scores: {'rouge-1': {'r': 0.4931050996492532, 'p': 0.5062838536207949, 'f': 0.48421650710726766}, 'rouge-2': {'r': 0.2867267329318735, 'p': 0.29386559356758024, 'f': 0.28114994067762966}, 'rouge-l': {'r': 0.4929319008833392, 'p': 0.5060474222029537, 'f': 0.4840231909599921}}
Exact Match (EM): 0.3650898770104068
F1 Score: 0.3650898770104068


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load a pre-trained model and tokenizer suitable for semantic similarity tasks
model_name = "textattack/roberta-base-MNLI"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

real_answers = reference_answers

semantic_similarity = pipeline("feature-extraction", model=model, tokenizer=tokenizer, device=0)
similarity_scores = []
for gen_ans, real_ans in zip(generated_answers, real_answers):


    # Compute embeddings
    with torch.no_grad():
        gen_embedding = semantic_similarity(gen_ans, return_tensors="pt").mean(axis=1)
        real_embedding = semantic_similarity(real_ans, return_tensors="pt").mean(axis=1)

    # Calculate cosine similarity
    similarity_score = cosine_similarity(gen_embedding, real_embedding)[0][0]
    similarity_scores.append(similarity_score)


# Overall similarity score
average_similarity = sum(similarity_scores) / len(similarity_scores)
print(f"Average Cosine Similarity Score: {average_similarity}")


Average Cosine Similarity Score: 0.8703485468612207
